# Distilling Knowledge in Multiple Students Using Generative Models

In [1]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [3]:
# Train the teacher model as usual
epochs = 20
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


Epoch 1/20
188/188 [==============================] - 4s 11ms/step - loss: 0.8719 - accuracy: 0.6807 - val_loss: 0.3312 - val_accuracy: 0.8777
Epoch 2/20
188/188 [==============================] - 2s 10ms/step - loss: 0.3500 - accuracy: 0.8756 - val_loss: 0.3218 - val_accuracy: 0.8813
Epoch 3/20
188/188 [==============================] - 2s 11ms/step - loss: 0.2820 - accuracy: 0.8993 - val_loss: 0.2854 - val_accuracy: 0.8932
Epoch 4/20
188/188 [==============================] - 2s 11ms/step - loss: 0.2463 - accuracy: 0.9121 - val_loss: 0.2439 - val_accuracy: 0.9113
Epoch 5/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2072 - accuracy: 0.9225 - val_loss: 0.2246 - val_accuracy: 0.9153
Epoch 6/20
188/188 [==============================] - 2s 9ms/step - loss: 0.1813 - accuracy: 0.9339 - val_loss: 0.2160 - val_accuracy: 0.9222
Epoch 7/20
188/188 [==============================] - 2s 9ms/step - loss: 0.1549 - accuracy: 0.9427 - val_loss: 0.2151 - val_accuracy: 0.9221
Ep

In [3]:
teacher.load_weights("Teacher_FMNIST_92.h5")

In [4]:
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255


teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 2s 2ms/step - loss: 0.3734 - accuracy: 0.9195


[0.351641982793808, 0.9239000082015991]

In [5]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 2 Students case
# ---------------------------------------------
s1Train=train_dense[:,:128]
s2Train=train_dense[:,128:]

s1Val=val_dense[:,:128]
s2Val=val_dense[:,128:]


In [7]:
def define_model(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(16, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(128,name='req'+name))
    student1.compile('adam',loss='mse',metrics=['accuracy'])
    student1.summary()
    return student1



In [8]:
s1=define_model("s1")
s2=define_model('s2')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 16)         

In [9]:
batch_size = 128
s1.fit([X_train],s1Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s1Val))

s2.fit([X_train],s2Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s2Val))

Epoch 1/10
375/375 [==============================] - 4s 9ms/step - loss: 1.0604 - accuracy: 0.0659 - val_loss: 0.4026 - val_accuracy: 0.3204
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.5122 - accuracy: 0.2885 - val_loss: 0.3108 - val_accuracy: 0.3848
Epoch 3/10
375/375 [==============================] - 4s 10ms/step - loss: 0.4497 - accuracy: 0.3208 - val_loss: 0.3183 - val_accuracy: 0.4047
Epoch 4/10
375/375 [==============================] - 3s 7ms/step - loss: 0.4229 - accuracy: 0.3311 - val_loss: 0.2797 - val_accuracy: 0.4116
Epoch 5/10
375/375 [==============================] - 2s 5ms/step - loss: 0.4045 - accuracy: 0.3433 - val_loss: 0.3041 - val_accuracy: 0.4099
Epoch 6/10
375/375 [==============================] - 3s 9ms/step - loss: 0.3983 - accuracy: 0.3447 - val_loss: 0.2620 - val_accuracy: 0.4108
Epoch 7/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3884 - accuracy: 0.3574 - val_loss: 0.2488 - val_accuracy: 0.4195
Epoch

### **2 Students**

In [10]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
output=tensorflow.keras.layers.concatenate([o1,o2])

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm2=Model([s1.get_layer("s1").input,s2.get_layer("s2").input], output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm2.get_layer('d1').set_weights(my_weights)

In [11]:
for l in mm2.layers[:len(mm2.layers)-1]:
    l.trainable=False

mm2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [12]:
batch_size = 128
mm2_history=mm2.fit([X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=30,
          verbose=1, validation_data=([X_val,X_val], Y_val))

Epoch 1/30
375/375 [==============================] - 2s 5ms/step - loss: 0.8066 - accuracy: 0.7410 - val_loss: 0.4641 - val_accuracy: 0.8145
Epoch 2/30
375/375 [==============================] - 3s 9ms/step - loss: 0.6623 - accuracy: 0.7456 - val_loss: 0.4590 - val_accuracy: 0.8112
Epoch 3/30
375/375 [==============================] - 4s 9ms/step - loss: 0.6214 - accuracy: 0.7553 - val_loss: 0.4596 - val_accuracy: 0.8083
Epoch 4/30
375/375 [==============================] - 4s 10ms/step - loss: 0.6019 - accuracy: 0.7611 - val_loss: 0.4608 - val_accuracy: 0.8158
Epoch 5/30
375/375 [==============================] - 2s 6ms/step - loss: 0.6007 - accuracy: 0.7629 - val_loss: 0.4573 - val_accuracy: 0.8144
Epoch 6/30
375/375 [==============================] - 2s 6ms/step - loss: 0.5946 - accuracy: 0.7662 - val_loss: 0.4574 - val_accuracy: 0.8096
Epoch 7/30
375/375 [==============================] - 4s 11ms/step - loss: 0.5955 - accuracy: 0.7624 - val_loss: 0.4568 - val_accuracy: 0.8142
Epoc

In [13]:
loss, acc = mm2.evaluate([X_test,X_test], Y_test, verbose=1)
loss, acc 

313/313 [==============================] - 2s 5ms/step - loss: 0.4759 - accuracy: 0.8012


(0.4759179651737213, 0.8011999726295471)